In [108]:
import sys
import os

In [109]:
class PDBEntry:
    def __init__(self, idx, name, chain):
        self.indexNum = idx
        self.name = name
        self.chain = chain

In [110]:
class single_AA:
    def __init__ (self, seqNum, chain, AA, SS_type):
        self.seqNum = seqNum
        self.chain = chain
        self.AA = AA
        self.SS_type = SS_type

In [111]:
class CSVAggregate:
    def __init__ (self, pdb_name, chain_id):
        self.pdb_name = pdb.name
        self.chain_id = chain_id
        self.PCASSO_AAs = []
        self.PDB_AAs = []
        
    def add_PCASSO_AA(self, seqNum, chain, AA, SS_type ):
        newAA = single_AA(seqNum, chain, AA, SS_type )
        self.PCASSO_AAs.append(newAA)
    
    def add_PDB_AA(self, seqNum, chain, AA, SS_type):
        newAA = single_AA(seqNum, chain, AA, SS_type )
        self.PDB_AAs.append(newAA) 

In [112]:
notebook_path = os.path.abspath("Analysis of CSV files.ipynb")

PDBList = os.path.join(os.path.dirname(notebook_path), "DataSet/Test_Set_300.csv")
PDBs = []
with open(PDBList) as file:
    for item in file:
        item = item.rstrip('\n')
        tmp = item.split(',')
        obj = PDBEntry(tmp[0], tmp[1],tmp[2])
        PDBs.append(obj)
print(len(PDBs))

300


In [113]:
CSVs = []
for pdb in PDBs:
    generic_path =  os.path.join(os.path.dirname(notebook_path),"PDBFiles",pdb.name)
    PCASSO_read_path = os.path.join(generic_path,pdb.name+'.csv')
    PDB_read_path = os.path.join(generic_path,pdb.name+'-pdb.csv')

    PCASSSO_in_file = open(PCASSO_read_path,'r')
    PDB_in_file = open(PDB_read_path,'r')
    lines = PCASSSO_in_file.readlines()
    name = lines[0]
    temp = CSVAggregate(name,pdb.chain)
    
    lines = lines[1:]
    for line in lines:
        parts = line.split(',')
        seqNum = int(parts[0])
        chain = parts[1]
        AA = parts[2]
        SS_type = parts[3][:1]
        temp.add_PCASSO_AA(seqNum, chain, AA, SS_type)
    
    PDB_lines = PDB_in_file.readlines()
    PDB_lines = PDB_lines[1:]
    for line in PDB_lines:
        parts = line.split(',')
        seqNum = int(parts[0])
        chain = parts[1]
        AA = parts[2]
        SS_type = parts[3][:1]
        temp.add_PDB_AA(seqNum, chain, AA, SS_type)    
    CSVs.append(temp)  
    temp = ''

In [114]:
global_true_helix = 0
global_true_sheet = 0
global_true_loop = 0
global_miss_H_as_S = 0
global_miss_H_as_L = 0
global_miss_S_as_H = 0
global_miss_S_as_L = 0
global_miss_L_as_S = 0
global_miss_L_as_H = 0
global_missing_AA = 0
global_PDB_H = 0
global_PDB_S = 0
global_PDB_L = 0
global_PCASSO_H = 0
global_PCASSO_S = 0
global_PCASSO_L = 0
weirdness = 0

In [ ]:
for csv in CSVs:
    local_true_helix = 0
    local_true_sheet = 0
    local_true_loop = 0
    local_miss_H_as_S = 0
    local_miss_H_as_L = 0
    local_miss_S_as_H = 0
    local_miss_S_as_L = 0
    local_miss_L_as_S = 0
    local_miss_L_as_H = 0
    local_missing_AA = 0
    local_mismatch = 0
    PDB_H = 0
    PDB_S = 0
    PDB_L = 0
    PCASSO_H = 0
    PCASSO_S = 0
    PCASSO_L = 0
    print(f'PDB {csv.pdb_name} chain {csv.chain_id}')
    for x in range(len(csv.PCASSO_AAs)):
        PCASSO_temp = csv.PCASSO_AAs[x]
        
        PDB_temp.seqNum = 'missing'
        PDB_temp.chain = ''
        PDB_temp.AA = ''
        PDB_temp.SS_type = 'missing'
        print(f'PCASSO SeqNum {PCASSO_temp.seqNum}')
        for k in range(len(csv.PDB_AAs)):
            print(f'k: {k} csv.PDB_AAs[k].seqNum{csv.PDB_AAs[k].seqNum})
#             print(f'pdb seq {csv.PDB_AAs[k].seqNum} PCASSO seq {PCASSO_temp.seqNum}')
            if int(csv.PDB_AAs[k].seqNum) == int(PCASSO_temp.seqNum):
                PDB_temp = csv.PDB_AAs[k]
                break
                
#         if PDB_temp.seqNum == 'missing':
#             local_missing_AA += 1  
        
        
#         if PDB_temp.seqNum != PCASSO_temp.seqNum:
#             print(f'x:{x} pdb seq {PDB_temp.seqNum} PCASSO seq {PCASSO_temp.seqNum}')
    
#     for x in range(len(csv.PCASSO_AAs)):
#         PCASSO_temp = csv.PCASSO_AAs[x]  
        
#         if x >= len(csv.PDB_AAs):
#             PDB_temp.seqNum = 'missing'
#             PDB_temp.chain = ''
#             PDB_temp.AA = ''
#             PDB_temp.SS_type = 'missing'
#             print('found missing')
#             local_missing_AA += 1            
#         else:
#             PDB_temp = csv.PDB_AAs[x]
                
#         print(f'x:{x} pdb seq {PDB_temp.seqNum} PCASSO seq {PCASSO_temp.seqNum}')
            
#         if csv.chain_id == PCASSO_temp.chain:
#             pdb_type = PDB_temp.SS_type
#             PCASSO_type = PCASSO_temp.SS_type
            
#             if PCASSO_type == 'H':
#                 PCASSO_H += 1
#             elif PCASSO_type == 'E':
#                 PCASSO_S += 1
#             elif PCASSO_type == 'C':
#                 PCASSO_L += 1
#             else:
#                 weirdness += 1
    
#             if pdb_type == 'H':
#                 PDB_H += 1
#             elif pdb_type == 'E':
#                 PDB_S += 1
#             elif pdb_type == 'C':
#                 PDB_L += 1
#             else:
#                 weirdness += 1
                
#             if pdb_type == 'H':
#                 if PCASSO_type == 'H':
#                     local_true_helix += 1
#                 elif PCASSO_type == 'E':
#                     local_miss_H_as_S += 1
#                 elif PCASSO_type == 'C':
#                     local_miss_H_as_L += 1
#                 else:
#                     weirdness += 1
                    
#             if pdb_type == 'E':
#                 if PCASSO_type == 'H':
#                     local_miss_S_as_H += 1
#                 elif PCASSO_type == 'E':
#                     local_true_sheet += 1
#                 elif PCASSO_type == 'C':
#                     local_miss_S_as_L += 1
#                 else:
#                     weirdness += 1
                    
#             if pdb_type == 'C':
#                 if PCASSO_type == 'H':
#                     local_miss_L_as_H += 1
#                 elif PCASSO_type == 'E':
#                     local_miss_L_as_S += 1
#                 elif PCASSO_type == 'C':
#                     local_true_loop += 1
#                 else:
#                     weirdness += 1
            
#             if pdb_type != PCASSO_type:
#                 print(f'miss pdb: {pdb_type} PCASSO: {PCASSO_type}')
#                 local_mismatch += 1
                
            
    
#     print(f'\nTrue helix {local_true_helix}, helix as sheet {local_miss_H_as_S}, helix as loop {local_miss_H_as_L}')
#     print(f'True sheet {local_true_sheet}, sheet as helix {local_miss_S_as_H},  sheet as loop {local_miss_S_as_L}')
#     print(f'True loop {local_true_loop}, loop as helix {local_miss_L_as_H}, loop as sheet {local_miss_L_as_S}\n')
    
#     total_miss = local_missing_AA + local_miss_H_as_S + local_miss_H_as_L + local_miss_S_as_H + local_miss_S_as_L + local_miss_L_as_H + local_miss_L_as_S
    
#     print(f'local mismatch direct = {local_mismatch}')
#     print(f'local mismatch calculated = {total_miss}')
#     print(f'missing AAs this protein {local_missing_AA}')
    
    
#     local_miss_H = local_miss_H_as_S + local_miss_H_as_L
#     local_miss_S = local_miss_S_as_H + local_miss_S_as_L
#     local_miss_L = local_miss_L_as_H + local_miss_L_as_S
    
#     if local_true_helix != 0:
#         accuracy_H = local_true_helix / (local_true_helix + local_miss_H)
#     else:
#         accuracy_H = 0
    
#     if local_true_sheet != 0:
#         accuracy_S = local_true_sheet / (local_true_sheet + local_miss_S)
#     else: 
#         accuracy_S = 0
        
#     if local_true_loop != 0:
#         accuracy_L = local_true_loop / (local_true_loop + local_miss_L)
#     else: 
#         accuracy_L = 0
        
#     print(f'Helix accuracy = {(accuracy_H * 100):.2f}')
#     print(f'Sheet accuracy = {(accuracy_S * 100):.2f}')
#     print(f'Loop accuracy = {(accuracy_L * 100):.2f}')
    
#     local_correct_pred = local_true_helix + local_true_sheet + local_true_loop
    
#     local_total_accuracy = local_correct_pred / (local_correct_pred + local_miss_H + local_miss_S + local_miss_L)
    
    
    
#     print(f'Total accuracy this protein {local_total_accuracy * 100:.2f}')
#     PDB_total = PDB_H + PDB_S + PDB_L
#     PCASSO_total = PCASSO_H + PCASSO_S + PCASSO_L
    
#     print(f'PDB_H {PDB_H} PDB_S {PDB_S} PDB_L {PDB_L} TOTAL {PDB_total}')
#     print(f'PCASSO_H {PCASSO_H} PCASSO_S {PCASSO_S} PCASSO_L {PCASSO_L} TOTAL {PCASSO_total}')
    
#     if PDB_total != PCASSO_total:
#         print(f'Mismatch! PDB_total: {PDB_total} != PCASSO_total: {PCASSO_total}')
#     print('\n\n')
    
#     global_missing_AA += local_missing_AA
    
#     global_true_helix += local_true_helix
#     global_true_sheet += local_true_sheet
#     global_true_loop += local_true_loop
          
#     global_miss_H_as_S += local_miss_H_as_S
#     global_miss_H_as_L += local_miss_H_as_L
#     global_miss_S_as_H += local_miss_S_as_H
#     global_miss_S_as_L += local_miss_S_as_L
#     global_miss_L_as_H += local_miss_L_as_H
#     global_miss_L_as_S += local_miss_L_as_S
    
#     global_PDB_H += PDB_H
#     global_PDB_S += PDB_S
#     global_PDB_L += PDB_L
#     global_PCASSO_H += PCASSO_H
#     global_PCASSO_S += PCASSO_S
#     global_PCASSO_L += PCASSO_L

In [119]:
print('---=== GLOBAL VALUES ===---')  
print(f'global missing AAs {global_missing_AA}\n')
global_miss_H = global_miss_H_as_S + global_miss_H_as_L
global_miss_S = global_miss_S_as_H + global_miss_S_as_L
global_miss_L = global_miss_L_as_H + global_miss_L_as_S
if global_true_helix != 0:
    global_accuracy_H = global_true_helix / (global_true_helix + global_miss_H)
else:
    global_accuracy_H = 0

if global_true_sheet != 0:
    global_accuracy_S = global_true_sheet / (global_true_sheet + global_miss_S)
else: 
   global_accuracy_S = 0
if global_true_loop != 0:
    global_accuracy_L = global_miss_L / global_true_loop# / (global_true_loop + global_miss_L)
else: 
    global_accuracy_L = 0
    
total_correct = global_true_helix + global_true_sheet + global_true_loop
total_missed = global_miss_H + global_miss_S + global_miss_L
global_total_accuracy = total_correct / (total_correct + total_missed)


print(f'True helix {global_true_helix}, helix as sheet {global_miss_H_as_S}, helix as loop {global_miss_H_as_L}')
print(f'Global Helix accuracy {(global_accuracy_H * 100):.2f}')
print(f'\nTrue sheet {global_true_sheet}, sheet as helix {global_miss_S_as_H},  sheet as loop {global_miss_S_as_L}')
print(f'Global Sheet accuracy {(global_accuracy_S * 100):.2f}')
print(f'\nTrue loop {global_true_loop}, loop as helix {global_miss_L_as_H}, loop as sheet {global_miss_L_as_S}')
print(f'Global Loop accuracy {(global_accuracy_L * 100):.2f}')
print(f'\nGlobal overall accuracy: {global_total_accuracy}')
# print(f'Weirdness {weirdness}')

print('\n')
print(f'GLOBAL PDB_H: {global_PDB_H:9} \t\tPDB_S: {global_PDB_S:8} \t\tPDB_L: {global_PDB_L:8} \t\tTOTAL: {global_PDB_H + global_PDB_S + global_PDB_L}')
print(f'GLOBAL PCASSO_H: {global_PCASSO_H:6} \t\tPCASSO_S {global_PCASSO_S:6} \t\tPCASSO_L: {global_PCASSO_L} \t\tTOTAL: {global_PCASSO_H + global_PCASSO_S + global_PCASSO_L}')


---=== GLOBAL VALUES ===---
global missing AAs 0



ZeroDivisionError: division by zero